# Logistic Regression
It predicts categorical responses when there are only two possible outcomes, such as:

- Predicting whether a credit card transaction is fradulent or not
- Predicting whether an individual will click into a website or not
- Predicting whether an individual will default on a loan or not

## Fitting Logistic Regression
In linear regression, the predicted response variable can take any value and is unconstrained. In logistic regression, however, the predicted response variable is limited to a probability between 0 and 1. 

To perform logistic regression mathematically:

- The categorical column labels (limited to two values) must be encoded with values of 0 and 1
- A linear model predicts the log odds instead of predicting the response itself
- Where p = the probability of a one value occurring, we can calculate the odds ratio

$$ log(\frac{p}{1-p}) = b_0+b_1x_1+b_2x_2+ ...$$

By taking the log of the odds ratio above and equating it to the linear regression line, the predictions will measure the probability of success from 0 to 1.

$$ p = \frac{e^{b_0+b_1x_1+b_2x_2+...}}{1+e^{b_0+b_1x_1+b_2x_2+...}}$$

The log of the odd ratio can be reformulated into the equation above to find the probability directly.

Steps for Fitting Logistic Regression in Python:

1. Load libraries. Similar to working with linear regression models, the statsmodels.api library can be used to run logistic regression models.
2. Use dummy variables for categorical columns
3. Impute missing values
4. Use statsmodels Logit() method and pass in the response variable, the intercept, and your explanatory variable, x.
5. Fit the model
6. Get summary results

#### Example:

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('support-files/16_Logistic-Regression/fraud_dataset.csv')
df.head()

,transaction_id,duration,day,fraud
0,28891,21.302600,weekend,False
1,61629,22.932765,weekend,False
2,53707,32.694992,weekday,False
3,47812,32.784252,weekend,False
4,43455,17.756828,weekend,False


`1.` As you can see, there are two columns that need to be changed to dummy variables.  Replace each of the current columns to the dummy version.  Use the 1 for `weekday` and `True`, and 0 otherwise.

In [2]:
# dummy variables for the 'day' column
day_dummies = pd.get_dummies(df['day'])
df = df.join(day_dummies)
df.head(1)

,transaction_id,duration,day,fraud,weekday,weekend
0,28891,21.3026,weekend,False,0,1


In [3]:
# dummy variables for the 'fraud' column
fraud_dummies = pd.get_dummies(df['fraud'])
df = df.join(fraud_dummies)
df.head(1)

,transaction_id,duration,day,fraud,weekday,weekend,False,True
0,28891,21.3026,weekend,False,0,1,1,0


In [4]:
# create intercept
df['intercept'] = 1

In [5]:
# drop useless columns
df.drop(columns=['weekend', False], inplace=True)
df.head(1)

,transaction_id,duration,day,fraud,weekday,True,intercept
0,28891,21.3026,weekend,False,0,0,1


In [6]:
# rename the 'True' column for clarity
df.rename(columns={True: "fraud", "fraud": "fraud-bool"}, inplace=True)
df.head(1)

,transaction_id,duration,day,fraud-bool,weekday,fraud,intercept
0,28891,21.3026,weekend,False,0,0,1


In [7]:
# checking to see if there are any missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8793 entries, 0 to 8792
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  8793 non-null   int64  
 1   duration        8793 non-null   float64
 2   day             8793 non-null   object 
 3   fraud-bool      8793 non-null   bool   
 4   weekday         8793 non-null   uint8  
 5   fraud           8793 non-null   uint8  
 6   intercept       8793 non-null   int64  
dtypes: bool(1), float64(1), int64(2), object(1), uint8(2)
memory usage: 300.7+ KB


In [8]:
# proportion of fraudulent transactions
df.query('fraud == 1').count()[0] / df.shape[0]

0.012168770612987604

In [9]:
# average duration for fradulent transactions
df.query('fraud == 1')['duration'].mean()

4.624247370615658

In [10]:
# average duration for non-fradulent transactions
df.query('fraud == 0')['duration'].mean()

30.013583132522587

In [11]:
# proportion of weekday transations
df.query('weekday == 1').count()[0] / df.shape[0]

0.3452746502900034

In [12]:
# relationship between weekday, fraud and mean duration of transation
df.groupby(['weekday', 'fraud'], as_index=False)['duration'].mean()

,weekday,fraud,duration
0,0,0,30.005684
1,0,1,4.602264
2,1,0,30.028887
3,1,1,4.632039


In [13]:
# relationship between weekday and mean fraud
df.groupby(['weekday'], as_index=False)['fraud'].mean()

,weekday,fraud
0,0,0.004864
1,1,0.026021


`2.` Now that you have dummy variables, fit a logistic regression model to predict if a transaction is fraud using both day and duration. Don't forget an intercept! Also remember to use the `.summary2()` method to get your summary results.

In [14]:
logit_mod = sm.Logit(df['fraud'], df[['intercept', 'duration', 'weekday']])
results = logit_mod.fit()
results.summary2()

Optimization terminated successfully.
         Current function value: inf
         Iterations 16


C:\Users\tharc\anaconda3\envs\dand\lib\site-packages\statsmodels\discrete\discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\Users\tharc\anaconda3\envs\dand\lib\site-packages\statsmodels\discrete\discrete_model.py:1872: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
C:\Users\tharc\anaconda3\envs\dand\lib\site-packages\statsmodels\discrete\discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\Users\tharc\anaconda3\envs\dand\lib\site-packages\statsmodels\discrete\discrete_model.py:1872: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
C:\Users\tharc\anaconda3\envs\dand\lib\site-packages\statsmodels\base\model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
C:\Users\tharc\anaconda3\env

<class 'statsmodels.iolib.summary2.Summary'>
"""
                       Results: Logit
=============================================================
Model:              Logit            Pseudo R-squared: inf   
Dependent Variable: fraud            AIC:              inf   
Date:               2022-04-24 20:28 BIC:              inf   
No. Observations:   8793             Log-Likelihood:   -inf  
Df Model:           2                LL-Null:          0.0000
Df Residuals:       8790             LLR p-value:      1.0000
Converged:          1.0000           Scale:            1.0000
No. Iterations:     16.0000                                  
-------------------------------------------------------------
               Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
-------------------------------------------------------------
intercept      9.8709   1.9438  5.0783 0.0000  6.0613 13.6806
duration      -1.4637   0.2905 -5.0389 0.0000 -2.0331 -0.8944
weekday        2.5465   0.9043  2.8160 0.0049  0.7741  4.3188
=============================================================

"""

## Interpreting Results
1. Quantitative interpretations for every one unit increase in the explanatory variable $x_1x$, we expect a multiplicative change in the odds of being in the 1 category of $e^{b_1}$, holding all over variables constant.

2. Categorical interpretations: When in category $x_1$, we expect a multiplicative change in the ods of a 1 by $e^{b_1}$ compared to the baseline.